# Cleaning Spotify Data

In this notebook, we investigate the dataset on 600k+ track collected by Yamac Eren Ay and hosted on Kaggle [here](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks). The dataset contains information on songs collected through the Spoitpy API for songs released between 1921 and 2020. Our main goal is to create a smaller subset of the data containing songs that appeared in the Billboard Hot 100s year end lists as well as an equal number of songs from the same time range that did not appear on the list. Let's get started!

### Importing the data
The dataset is contained in the "tracks.csv" file in the repository. We first load the dataset using the appropriate libraries.

In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns

Now let's import the data. A point of note is that we will first transform the artists column into a list of lists where each observation contains a list of artists for the given song. To do this, we use the `literal_eval` function imported above.

In [3]:
tracks = pd.read_csv('tracks.csv', parse_dates = ['release_date'], converters={'artists': literal_eval})
tracks.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,[Uli],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,[Fernando Pessoa],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,[Dick Haymes],['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,[Dick Haymes],['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,[Francis Marty],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922-01-01,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,[Mistinguett],['4AxgXfD7ISvJSTObqm4aIE'],1922-01-01,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,[Greg Fieler],['5nWlsH5RDgFuRAiDeOFVmf'],1922-01-01,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


We need a way to parse out the names of the first artists in the dataframe from the list in the `artists` column. To do this, we make use of _lambda functions_. We also create a feature that tracks the song's length in minutes.

In [4]:
# Getting the main artist's name and also getting the 
tracks.loc[:,'main_artist'] = tracks.artists.map(lambda x: x[0])
tracks['duration_min'] = tracks['duration_ms'] / 60000

# Getting the dataset head
tracks.head(100)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,main_artist,duration_min
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,[Uli],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,...,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,Uli,2.115050
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,[Fernando Pessoa],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,...,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,Fernando Pessoa,1.636667
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,...,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,Ignacio Corsini,3.027333
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,...,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,Ignacio Corsini,2.948450
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,[Dick Haymes],['3BiJGZsyX9sJchTqcSA7Su'],1922-01-01,0.402,0.1580,...,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,Dick Haymes,2.718000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4CjnVzvSu86vqI5gY8SjPx,Elle Vendait Des P'tits Gateaux,0,151573,0,[Félix Mayol],['7DIlOK9L8d0IQ7Xk8aJxDW'],1922-01-01,0.716,0.3010,...,1,0.1050,0.987,0.0000,0.277,0.969,129.153,4,Félix Mayol,2.526217
96,4DE7VBCgyTjqV0Plc9Ra0U,Hush-A-Bye (Wee Rose of Kilarney),0,156880,0,"[Dick Haymes, Victor Young, His Orchestra]","['3BiJGZsyX9sJchTqcSA7Su', '3HqN7Sq7rmpOEI9UV5...",1922-01-01,0.279,0.1280,...,1,0.0406,0.994,0.6130,0.116,0.145,79.121,5,Dick Haymes,2.614667
97,4FCWuuqM851ACz8p1b41fi,Capítulo 2.23 - Banquero Anarquista,0,97200,0,[Fernando Pessoa],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.714,0.2190,...,0,0.9540,0.520,0.0000,0.222,0.540,110.822,3,Fernando Pessoa,1.620000
98,4RR5RKLA2vTQxkWrMIeHem,Cap Polonio - Remasterizado,0,161080,0,[Ignacio Corsini],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.541,0.1610,...,1,0.0940,0.996,0.9070,0.129,0.708,143.105,4,Ignacio Corsini,2.684667


Naturally this is a huge dataset. Let's trim it down a little to only include non-duplicate songs

In [5]:
# Dropping all duplicate values based on name and artist names
tracks.drop_duplicates(subset=['name','main_artist'], keep='first', inplace=True)

# Describing the data
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 523466 entries, 0 to 586670
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                523466 non-null  object        
 1   name              523465 non-null  object        
 2   popularity        523466 non-null  int64         
 3   duration_ms       523466 non-null  int64         
 4   explicit          523466 non-null  int64         
 5   artists           523466 non-null  object        
 6   id_artists        523466 non-null  object        
 7   release_date      523466 non-null  datetime64[ns]
 8   danceability      523466 non-null  float64       
 9   energy            523466 non-null  float64       
 10  key               523466 non-null  int64         
 11  loudness          523466 non-null  float64       
 12  mode              523466 non-null  int64         
 13  speechiness       523466 non-null  float64       
 14  acou

Let's further restrict our data to just songs that were released post 2008.

In [6]:
from datetime import datetime
tracks_08 = tracks.loc[tracks.release_date >= datetime(2008,1,1)].copy()
tracks_08.sort_values(by=['release_date'], inplace=True)
tracks_08.reset_index(drop=True, inplace=True)
print(tracks_08.info())
tracks_08.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127960 entries, 0 to 127959
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                127960 non-null  object        
 1   name              127960 non-null  object        
 2   popularity        127960 non-null  int64         
 3   duration_ms       127960 non-null  int64         
 4   explicit          127960 non-null  int64         
 5   artists           127960 non-null  object        
 6   id_artists        127960 non-null  object        
 7   release_date      127960 non-null  datetime64[ns]
 8   danceability      127960 non-null  float64       
 9   energy            127960 non-null  float64       
 10  key               127960 non-null  int64         
 11  loudness          127960 non-null  float64       
 12  mode              127960 non-null  int64         
 13  speechiness       127960 non-null  float64       
 14  acou

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,main_artist,duration_min
0,7djl4lZ6guZYAckDsfzrfs,Reckoning Song,57,166813,0,"[Asaf Avidan, The Mojos]","['7t51dSX8ZkKC7VoKRd0lME', '5abaNg0SJPEF7EST0R...",2008-01-01,0.610,0.285,...,0,0.2040,0.78100,0.000000,0.0769,0.278,106.092,4,Asaf Avidan,2.780217
1,5tRvqUPv7M4qqaDctwc0kQ,Préstame Tu Piel,45,215587,0,[División Minúscula],['2Ky9mFKNApb9KpEcORXE3p'],2008-01-01,0.463,0.446,...,1,0.0275,0.67100,0.000000,0.1180,0.452,167.395,4,División Minúscula,3.593117
2,3v03GNovV38IjeiXxg3p8K,El Cid,47,158053,0,[Colmillo Norteño],['5YvrXxWXf9n9enAV8HHZSN'],2008-01-01,0.731,0.925,...,1,0.1070,0.56200,0.000000,0.2500,0.942,170.091,3,Colmillo Norteño,2.634217
3,75tYBL0pbI5JCWhRN57Hb9,"Tan Fuerte, Tan Frágil",45,221280,0,[División Minúscula],['2Ky9mFKNApb9KpEcORXE3p'],2008-01-01,0.507,0.908,...,0,0.0391,0.00372,0.000000,0.1990,0.866,157.045,4,División Minúscula,3.688000
4,5GHaftqtPACEc7qiAf6wfR,Quinto Piso,43,301493,0,[Ricardo Arjona],['0h1zs4CTlU9D2QtgPxptUD'],2008-01-01,0.433,0.536,...,0,0.0327,0.51600,0.000003,0.3480,0.133,136.186,4,Ricardo Arjona,5.024883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0dN00URbdRTY8ul0zkO0W3,Step Into the Sunshine,45,303333,0,[Gary B],['3j6HqaYmvmWOTF1MH7YTYk'],2008-01-01,0.547,0.814,...,1,0.0294,0.02200,0.080700,0.1230,0.526,91.999,4,Gary B,5.055550
96,7rtsb0nMmWilPcw6mkRXnU,Alma En Libertad,32,270200,0,[Lilly Goodman],['6vetaGijEBK3wfhtCUWRBS'],2008-01-01,0.753,0.803,...,0,0.0845,0.16400,0.000178,0.1040,0.913,90.034,4,Lilly Goodman,4.503333
97,0fEuIosdE4UTHACOrmjR1f,Alkohol,50,269413,0,[Herbert Grönemeyer],['5tG5dl5mS2wB5w72YBpsyg'],2008-01-01,0.859,0.874,...,0,0.0322,0.34300,0.000000,0.4150,0.966,116.321,4,Herbert Grönemeyer,4.490217
98,48jZRDondCHOUuvTxpqU6F,Toma Mis Manos,24,363720,0,[Willie Colón],['7x5Slu7yTE5icZjNsc3OzW'],2008-01-01,0.480,0.657,...,0,0.0421,0.52500,0.002330,0.2750,0.688,97.785,4,Willie Colón,6.062000


### Extracting the Hot 100 Songs
We now import the Hot 100 Dataset we created in the Data Collection Notebook earlier. Recall it is in the `hot_100_lyrics.csv` file.

In [8]:
hot100 = pd.read_csv('hot_100_lyrics.csv')
hot100.head(100)

,title,artists,rank,year,first_artist,title_fmtd,lyrics
0,TiK ToK,Ke$ha,1,2010,kesha,tik tok,Wake up in the morning feelin' like P. Diddy @...
1,Need You Now,Lady Antebellum,2,2010,lady antebellum,need you now,[Intro: Hillary Scott & @ Charles Kelley @ ] ...
2,"Hey, Soul Sister",Train,3,2010,train,hey soul sister,Heyy @ He-e-e-e-ey @ He-e-e-e-ey @ Your lipsti...
3,California Gurls,Katy Perry Featuring Snoop Dogg,4,2010,katy perry,california gurls,"Greetings, loved ones @ Let's take a journey @..."
4,OMG,Usher Featuring will.i.am,5,2010,usher,omg,"Oh my gosh @ Baby let me @ I did it again, so ..."
...,...,...,...,...,...,...,...
95,Life After You,Daughtry,96,2010,daughtry,life after you,Ten miles from town and I just broke down @ Sp...
96,Smile,Uncle Kracker,97,2010,uncle kracker,smile,You're better than the best @ I'm lucky just t...
97,Teach Me How To Dougie,Cali Swag District,98,2010,cali swag district,teach me how to dougie,"They be like ""Smoove Can you teach me how to ..."
98,Try Sleeping With A Broken Heart,Alicia Keys,99,2010,alicia keys,try sleeping with a broken heart,"Let's do it, baby, let's do it, baby @ Let's d..."


Okay great! Now we need to figure out a way to create a dataframe containing the songs from the billboard hot 100s, but including the extra spotify data. I am going to go over a very naive (and probably very inefficient) way to do this by iterating over all the songs in the `tracks_08` dataframe and seeing if they match any instance of the songs in the `hot100` dataframe. We need to take care of formatting but this is just a first pass for now. If there's a 'hit', it marks the observation as True, otherwise it will be False. We can then index based off of these True and False values!
__Note:__ The loop took an obscenely long amount of time on my laptop (around 90 minutes) and still didn't return a full list of the songs.

In [9]:
# First creating a default column that says each song is not on the hot100 list
tracks_08['hot100'] = False

# Now creating a massive loop!
# Outer loop is going over tracks in the tracks_08 dataframe
for song in range(len(tracks_08)):
    print("Song", song)
    # Recording the title and main artist names
    title = tracks_08.loc[song, 'name'].lower().strip()
    artist = tracks_08.loc[song, 'main_artist'].lower().strip()
    
    # Now looping through each individual hot100 track
    for hot in range(len(hot100)):
        # Recording the names
        hot_title = hot100.loc[hot, 'title'].lower().strip()
        hot_title_fmtd = hot100.loc[hot, 'title_fmtd'].lower().strip()
        hot_artist = hot100.loc[hot, 'artists'].lower().strip()
        hot_artist_fmtd = hot100.loc[hot, 'first_artist'].lower().strip()
        
        # Artist check
        check1 = (artist in hot_artist) or (hot_artist_fmtd in artist)
        # Title check
        check2 = (title == hot_title) or (title == hot_title_fmtd)
        
        # Now making a break
        if check1 and check2:
            tracks_08.loc[song, 'hot100'] = True
            break


Song 0
Song 1
Song 2
Song 3
Song 4
Song 5
Song 6
Song 7
Song 8
Song 9
Song 10
Song 11
Song 12
Song 13
Song 14
Song 15
Song 16
Song 17
Song 18
Song 19
Song 20
Song 21
Song 22
Song 23
Song 24
Song 25
Song 26
Song 27
Song 28
Song 29
Song 30
Song 31
Song 32
Song 33
Song 34
Song 35
Song 36
Song 37
Song 38
Song 39
Song 40
Song 41
Song 42
Song 43
Song 44
Song 45
Song 46
Song 47
Song 48
Song 49
Song 50
Song 51
Song 52
Song 53
Song 54
Song 55
Song 56
Song 57
Song 58
Song 59
Song 60
Song 61
Song 62
Song 63
Song 64
Song 65
Song 66
Song 67
Song 68
Song 69
Song 70
Song 71
Song 72
Song 73
Song 74
Song 75
Song 76
Song 77
Song 78
Song 79
Song 80
Song 81
Song 82
Song 83
Song 84
Song 85
Song 86
Song 87
Song 88
Song 89
Song 90
Song 91
Song 92
Song 93
Song 94
Song 95
Song 96
Song 97
Song 98
Song 99
Song 100
Song 101
Song 102
Song 103
Song 104
Song 105
Song 106
Song 107
Song 108
Song 109
Song 110
Song 111
Song 112
Song 113
Song 114
Song 115
Song 116
Song 117
Song 118
Song 119
Song 120
Song 121
Song 122
Son

Song 927
Song 928
Song 929
Song 930
Song 931
Song 932
Song 933
Song 934
Song 935
Song 936
Song 937
Song 938
Song 939
Song 940
Song 941
Song 942
Song 943
Song 944
Song 945
Song 946
Song 947
Song 948
Song 949
Song 950
Song 951
Song 952
Song 953
Song 954
Song 955
Song 956
Song 957
Song 958
Song 959
Song 960
Song 961
Song 962
Song 963
Song 964
Song 965
Song 966
Song 967
Song 968
Song 969
Song 970
Song 971
Song 972
Song 973
Song 974
Song 975
Song 976
Song 977
Song 978
Song 979
Song 980
Song 981
Song 982
Song 983
Song 984
Song 985
Song 986
Song 987
Song 988
Song 989
Song 990
Song 991
Song 992
Song 993
Song 994
Song 995
Song 996
Song 997
Song 998
Song 999
Song 1000
Song 1001
Song 1002
Song 1003
Song 1004
Song 1005
Song 1006
Song 1007
Song 1008
Song 1009
Song 1010
Song 1011
Song 1012
Song 1013
Song 1014
Song 1015
Song 1016
Song 1017
Song 1018
Song 1019
Song 1020
Song 1021
Song 1022
Song 1023
Song 1024
Song 1025
Song 1026
Song 1027
Song 1028
Song 1029
Song 1030
Song 1031
Song 1032
Song 1033
Son

Song 1754
Song 1755
Song 1756
Song 1757
Song 1758
Song 1759
Song 1760
Song 1761
Song 1762
Song 1763
Song 1764
Song 1765
Song 1766
Song 1767
Song 1768
Song 1769
Song 1770
Song 1771
Song 1772
Song 1773
Song 1774
Song 1775
Song 1776
Song 1777
Song 1778
Song 1779
Song 1780
Song 1781
Song 1782
Song 1783
Song 1784
Song 1785
Song 1786
Song 1787
Song 1788
Song 1789
Song 1790
Song 1791
Song 1792
Song 1793
Song 1794
Song 1795
Song 1796
Song 1797
Song 1798
Song 1799
Song 1800
Song 1801
Song 1802
Song 1803
Song 1804
Song 1805
Song 1806
Song 1807
Song 1808
Song 1809
Song 1810
Song 1811
Song 1812
Song 1813
Song 1814
Song 1815
Song 1816
Song 1817
Song 1818
Song 1819
Song 1820
Song 1821
Song 1822
Song 1823
Song 1824
Song 1825
Song 1826
Song 1827
Song 1828
Song 1829
Song 1830
Song 1831
Song 1832
Song 1833
Song 1834
Song 1835
Song 1836
Song 1837
Song 1838
Song 1839
Song 1840
Song 1841
Song 1842
Song 1843
Song 1844
Song 1845
Song 1846
Song 1847
Song 1848
Song 1849
Song 1850
Song 1851
Song 1852
Song 1853


Song 2579
Song 2580
Song 2581
Song 2582
Song 2583
Song 2584
Song 2585
Song 2586
Song 2587
Song 2588
Song 2589
Song 2590
Song 2591
Song 2592
Song 2593
Song 2594
Song 2595
Song 2596
Song 2597
Song 2598
Song 2599
Song 2600
Song 2601
Song 2602
Song 2603
Song 2604
Song 2605
Song 2606
Song 2607
Song 2608
Song 2609
Song 2610
Song 2611
Song 2612
Song 2613
Song 2614
Song 2615
Song 2616
Song 2617
Song 2618
Song 2619
Song 2620
Song 2621
Song 2622
Song 2623
Song 2624
Song 2625
Song 2626
Song 2627
Song 2628
Song 2629
Song 2630
Song 2631
Song 2632
Song 2633
Song 2634
Song 2635
Song 2636
Song 2637
Song 2638
Song 2639
Song 2640
Song 2641
Song 2642
Song 2643
Song 2644
Song 2645
Song 2646
Song 2647
Song 2648
Song 2649
Song 2650
Song 2651
Song 2652
Song 2653
Song 2654
Song 2655
Song 2656
Song 2657
Song 2658
Song 2659
Song 2660
Song 2661
Song 2662
Song 2663
Song 2664
Song 2665
Song 2666
Song 2667
Song 2668
Song 2669
Song 2670
Song 2671
Song 2672
Song 2673
Song 2674
Song 2675
Song 2676
Song 2677
Song 2678


Song 3405
Song 3406
Song 3407
Song 3408
Song 3409
Song 3410
Song 3411
Song 3412
Song 3413
Song 3414
Song 3415
Song 3416
Song 3417
Song 3418
Song 3419
Song 3420
Song 3421
Song 3422
Song 3423
Song 3424
Song 3425
Song 3426
Song 3427
Song 3428
Song 3429
Song 3430
Song 3431
Song 3432
Song 3433
Song 3434
Song 3435
Song 3436
Song 3437
Song 3438
Song 3439
Song 3440
Song 3441
Song 3442
Song 3443
Song 3444
Song 3445
Song 3446
Song 3447
Song 3448
Song 3449
Song 3450
Song 3451
Song 3452
Song 3453
Song 3454
Song 3455
Song 3456
Song 3457
Song 3458
Song 3459
Song 3460
Song 3461
Song 3462
Song 3463
Song 3464
Song 3465
Song 3466
Song 3467
Song 3468
Song 3469
Song 3470
Song 3471
Song 3472
Song 3473
Song 3474
Song 3475
Song 3476
Song 3477
Song 3478
Song 3479
Song 3480
Song 3481
Song 3482
Song 3483
Song 3484
Song 3485
Song 3486
Song 3487
Song 3488
Song 3489
Song 3490
Song 3491
Song 3492
Song 3493
Song 3494
Song 3495
Song 3496
Song 3497
Song 3498
Song 3499
Song 3500
Song 3501
Song 3502
Song 3503
Song 3504


...

Song 125073
Song 125074
Song 125075
Song 125076
Song 125077
Song 125078
Song 125079
Song 125080
Song 125081
Song 125082
Song 125083
Song 125084
Song 125085
Song 125086
Song 125087
Song 125088
Song 125089
Song 125090
Song 125091
Song 125092
Song 125093
Song 125094
Song 125095
Song 125096
Song 125097
Song 125098
Song 125099
Song 125100
Song 125101
Song 125102
Song 125103
Song 125104
Song 125105
Song 125106
Song 125107
Song 125108
Song 125109
Song 125110
Song 125111
Song 125112
Song 125113
Song 125114
Song 125115
Song 125116
Song 125117
Song 125118
Song 125119
Song 125120
Song 125121
Song 125122
Song 125123
Song 125124
Song 125125
Song 125126
Song 125127
Song 125128
Song 125129
Song 125130
Song 125131
Song 125132
Song 125133
Song 125134
Song 125135
Song 125136
Song 125137
Song 125138
Song 125139
Song 125140
Song 125141
Song 125142
Song 125143
Song 125144
Song 125145
Song 125146
Song 125147
Song 125148
Song 125149
Song 125150
Song 125151
Song 125152
Song 125153
Song 125154
Song 125155
Song

Song 125759
Song 125760
Song 125761
Song 125762
Song 125763
Song 125764
Song 125765
Song 125766
Song 125767
Song 125768
Song 125769
Song 125770
Song 125771
Song 125772
Song 125773
Song 125774
Song 125775
Song 125776
Song 125777
Song 125778
Song 125779
Song 125780
Song 125781
Song 125782
Song 125783
Song 125784
Song 125785
Song 125786
Song 125787
Song 125788
Song 125789
Song 125790
Song 125791
Song 125792
Song 125793
Song 125794
Song 125795
Song 125796
Song 125797
Song 125798
Song 125799
Song 125800
Song 125801
Song 125802
Song 125803
Song 125804
Song 125805
Song 125806
Song 125807
Song 125808
Song 125809
Song 125810
Song 125811
Song 125812
Song 125813
Song 125814
Song 125815
Song 125816
Song 125817
Song 125818
Song 125819
Song 125820
Song 125821
Song 125822
Song 125823
Song 125824
Song 125825
Song 125826
Song 125827
Song 125828
Song 125829
Song 125830
Song 125831
Song 125832
Song 125833
Song 125834
Song 125835
Song 125836
Song 125837
Song 125838
Song 125839
Song 125840
Song 125841
Song

Song 126445
Song 126446
Song 126447
Song 126448
Song 126449
Song 126450
Song 126451
Song 126452
Song 126453
Song 126454
Song 126455
Song 126456
Song 126457
Song 126458
Song 126459
Song 126460
Song 126461
Song 126462
Song 126463
Song 126464
Song 126465
Song 126466
Song 126467
Song 126468
Song 126469
Song 126470
Song 126471
Song 126472
Song 126473
Song 126474
Song 126475
Song 126476
Song 126477
Song 126478
Song 126479
Song 126480
Song 126481
Song 126482
Song 126483
Song 126484
Song 126485
Song 126486
Song 126487
Song 126488
Song 126489
Song 126490
Song 126491
Song 126492
Song 126493
Song 126494
Song 126495
Song 126496
Song 126497
Song 126498
Song 126499
Song 126500
Song 126501
Song 126502
Song 126503
Song 126504
Song 126505
Song 126506
Song 126507
Song 126508
Song 126509
Song 126510
Song 126511
Song 126512
Song 126513
Song 126514
Song 126515
Song 126516
Song 126517
Song 126518
Song 126519
Song 126520
Song 126521
Song 126522
Song 126523
Song 126524
Song 126525
Song 126526
Song 126527
Song

Song 127133
Song 127134
Song 127135
Song 127136
Song 127137
Song 127138
Song 127139
Song 127140
Song 127141
Song 127142
Song 127143
Song 127144
Song 127145
Song 127146
Song 127147
Song 127148
Song 127149
Song 127150
Song 127151
Song 127152
Song 127153
Song 127154
Song 127155
Song 127156
Song 127157
Song 127158
Song 127159
Song 127160
Song 127161
Song 127162
Song 127163
Song 127164
Song 127165
Song 127166
Song 127167
Song 127168
Song 127169
Song 127170
Song 127171
Song 127172
Song 127173
Song 127174
Song 127175
Song 127176
Song 127177
Song 127178
Song 127179
Song 127180
Song 127181
Song 127182
Song 127183
Song 127184
Song 127185
Song 127186
Song 127187
Song 127188
Song 127189
Song 127190
Song 127191
Song 127192
Song 127193
Song 127194
Song 127195
Song 127196
Song 127197
Song 127198
Song 127199
Song 127200
Song 127201
Song 127202
Song 127203
Song 127204
Song 127205
Song 127206
Song 127207
Song 127208
Song 127209
Song 127210
Song 127211
Song 127212
Song 127213
Song 127214
Song 127215
Song

Song 127817
Song 127818
Song 127819
Song 127820
Song 127821
Song 127822
Song 127823
Song 127824
Song 127825
Song 127826
Song 127827
Song 127828
Song 127829
Song 127830
Song 127831
Song 127832
Song 127833
Song 127834
Song 127835
Song 127836
Song 127837
Song 127838
Song 127839
Song 127840
Song 127841
Song 127842
Song 127843
Song 127844
Song 127845
Song 127846
Song 127847
Song 127848
Song 127849
Song 127850
Song 127851
Song 127852
Song 127853
Song 127854
Song 127855
Song 127856
Song 127857
Song 127858
Song 127859
Song 127860
Song 127861
Song 127862
Song 127863
Song 127864
Song 127865
Song 127866
Song 127867
Song 127868
Song 127869
Song 127870
Song 127871
Song 127872
Song 127873
Song 127874
Song 127875
Song 127876
Song 127877
Song 127878
Song 127879
Song 127880
Song 127881
Song 127882
Song 127883
Song 127884
Song 127885
Song 127886
Song 127887
Song 127888
Song 127889
Song 127890
Song 127891
Song 127892
Song 127893
Song 127894
Song 127895
Song 127896
Song 127897
Song 127898
Song 127899
Song

Now that we have (painstakingly) finished the loop, let's get the data for the hot100!

In [18]:
tracks_hot100 = tracks_08[tracks_08.hot100 == True].copy()
tracks_hot100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796 entries, 905 to 127953
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                796 non-null    object        
 1   name              796 non-null    object        
 2   popularity        796 non-null    int64         
 3   duration_ms       796 non-null    int64         
 4   explicit          796 non-null    int64         
 5   artists           796 non-null    object        
 6   id_artists        796 non-null    object        
 7   release_date      796 non-null    datetime64[ns]
 8   danceability      796 non-null    float64       
 9   energy            796 non-null    float64       
 10  key               796 non-null    int64         
 11  loudness          796 non-null    float64       
 12  mode              796 non-null    int64         
 13  speechiness       796 non-null    float64       
 14  acousticness      796

In [19]:
tracks_hot100.to_csv('tracks_hot100.csv', index=False)

Looks like the loop didn't parse through the entire list since we have only 829 elements (although I am not sure how many are even duplicates). But, we can still work with this! Now let's try to extract the non-hot100 dataset.

### Not 100 Dataset

In [11]:
tracks_not100 = tracks_08.copy().drop(index=tracks_hot100.index)
tracks_not100.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127164 entries, 0 to 127959
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                127164 non-null  object        
 1   name              127164 non-null  object        
 2   popularity        127164 non-null  int64         
 3   duration_ms       127164 non-null  int64         
 4   explicit          127164 non-null  int64         
 5   artists           127164 non-null  object        
 6   id_artists        127164 non-null  object        
 7   release_date      127164 non-null  datetime64[ns]
 8   danceability      127164 non-null  float64       
 9   energy            127164 non-null  float64       
 10  key               127164 non-null  int64         
 11  loudness          127164 non-null  float64       
 12  mode              127164 non-null  int64         
 13  speechiness       127164 non-null  float64       
 14  acou

Let's try to cut down the dataset to only include a subsample of songs from 2010 onwards.

In [12]:
tracks_not100 = tracks_not100.loc[tracks_not100.release_date >= datetime(2010,1,1)].copy()

In [13]:
tracks_not100.sort_values(by=['release_date'], inplace=True)
tracks_not100.reset_index(drop=True, inplace=True)
print(tracks_not100.info())
tracks_not100.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110581 entries, 0 to 110580
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                110581 non-null  object        
 1   name              110581 non-null  object        
 2   popularity        110581 non-null  int64         
 3   duration_ms       110581 non-null  int64         
 4   explicit          110581 non-null  int64         
 5   artists           110581 non-null  object        
 6   id_artists        110581 non-null  object        
 7   release_date      110581 non-null  datetime64[ns]
 8   danceability      110581 non-null  float64       
 9   energy            110581 non-null  float64       
 10  key               110581 non-null  int64         
 11  loudness          110581 non-null  float64       
 12  mode              110581 non-null  int64         
 13  speechiness       110581 non-null  float64       
 14  acou

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,main_artist,duration_min,hot100
0,5tJ10vb1mkkGqkJeouAzlx,Chocabeck,41,282173,0,[Zucchero],['2KftmGt9sk1yLjsAoloC3M'],2010-01-01,0.584,0.961,...,0.0686,0.43400,0.003290,0.210,0.577,135.006,4,Zucchero,4.702883,False
1,7LRYw74nztDptYIStswo9K,Ya No Te Buscaré,58,191067,0,[La Arrolladora Banda El Limón De Rene Camacho],['5bSfBBCxY8QAk4Pifveisz'],2010-01-01,0.703,0.566,...,0.0274,0.26700,0.000102,0.163,0.824,129.919,4,La Arrolladora Banda El Limón De Rene Camacho,3.184450,False
2,211K0lo2zDojSRPCkW5KJc,Mi Amante,30,212640,0,[Valentín Elizalde],['3CAhiUHkUYT1mFtVHM9SHA'],2010-01-01,0.801,0.249,...,0.0499,0.75700,0.000001,0.214,0.965,95.962,3,Valentín Elizalde,3.544000,False
3,7BwxEQytlMM2VXzSWd0dJo,Mi Pais,37,260827,0,[Ricardo Arjona],['0h1zs4CTlU9D2QtgPxptUD'],2010-01-01,0.527,0.322,...,0.0580,0.89300,0.000000,0.108,0.268,135.506,4,Ricardo Arjona,4.347117,False
4,3z613YwTlIIDulcfPRp6GE,La jument de Michao,43,176333,0,[Nolwenn Leroy],['6suRVbMTiFWlTpB6fdLdF2'],2010-01-01,0.573,0.782,...,0.0560,0.29100,0.000000,0.167,0.520,104.084,4,Nolwenn Leroy,2.938883,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0KjEYmw0KaSg3uXS0GU7un,Gaminart,47,309734,1,"[Crack Family, Aerophon]","['7zXRgSpjJkQmB9SnPLq1Qk', '6nfNQvqechLUJaFHYA...",2010-01-01,0.814,0.428,...,0.0768,0.01730,0.000000,0.196,0.683,93.014,4,Crack Family,5.162233,False
96,7BrhAw4AnT8p62zqhCSW6O,アニマロッサ,36,273573,0,[PornoGraffitti],['5jQa0f5MIMPEK7udL7UpI6'],2010-01-01,0.552,0.967,...,0.0552,0.01660,0.000000,0.295,0.653,117.063,4,PornoGraffitti,4.559550,False
97,6Y7CXp4HBvs06CoDaDXW1o,Ewakuacja,41,197907,0,[Ewa Farna],['6xajh3A5qhxsNffhhBNntC'],2010-01-01,0.383,0.900,...,0.1240,0.00584,0.000000,0.306,0.359,159.763,4,Ewa Farna,3.298450,False
98,4yNaKJadMRE7bszUBcJJqd,Suratan Takdir,42,230133,0,[Gersang],['2rB8IK11fIwAclbh7264OU'],2010-01-01,0.344,0.796,...,0.0346,0.04000,0.000896,0.331,0.408,71.295,4,Gersang,3.835550,False


Now that we have the main dataset, let's start to sample (year-by-year) 100 songs and add them to a new dataframe! To do this, I will first create an empty dataframe and then keep appending a subsample for each year to it.

In [14]:
# Making the empty dataframe
tracks_not100_sub = pd.DataFrame(columns=tracks_not100.columns.tolist())
print(tracks_not100_sub)

Empty DataFrame
Columns: [id, name, popularity, duration_ms, explicit, artists, id_artists, release_date, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, main_artist, duration_min, hot100]
Index: []

[0 rows x 23 columns]


In [15]:
# Making the loop
for i in range(2010, 2021, 1):
    print("Year", i)
    tracks_year = tracks_not100[(tracks_not100.release_date <= datetime(i,12,31)) & (tracks_not100.release_date >= datetime(i,1,3))].copy()
    subsample = tracks_year.sample(n=100, random_state=440)
    print(subsample.head())
    tracks_not100_sub = tracks_not100_sub.append(subsample, ignore_index=True)

Year 2010
                          id                    name  popularity  duration_ms  \
3197  6GQLX6Z28fYwDNCrhaKzYF       Undercover Martyn          68       166573   
3475  6O4QbnxKqUCJMIGJe6rTaN   Pirouette, cacahouète          44       171147   
8008  04J24gXLrUEYyNmmoJMLNa       Make A Circuit Me           0       176427   
5046  1PdnoVxzvexOoLkOaTq7KX                 Poupata          29       214427   
7092  3DOOQAScRTa3oTszGN3XZg  Todo vuelve a su lugar          48       299973   

      explicit                                  artists  \
3197         0                   [Two Door Cinema Club]   
3475         0                  [Chansons Et Comptines]   
8008         0                           [The Polecats]   
5046         0  [Michal David, Kroky Frantiska Janecka]   
7092         0                   [Fidel Nadal, I Nesta]   

                                             id_artists release_date  \
3197                         ['536BYVgOnRky0xjsPT96zl']   2010-03-01   
3475

Year 2019
                           id                                   name  \
86542  2pSOoP6FLJWeRBxdvhyvkJ                                  Presa   
86436  6q8YtmvckE9yOtfQ9W6WUP                        Heaven Can Wait   
88091  6AomCkEOQUODCYzQwH4SWC                            Floor Seats   
89074  1UpP7aPDapP71MYrTQ8yTU  Cos'è l'amore (feat. Franco Califano)   
86177  0q3QoeqUKLIiHz7d43nvGb                        Piededzini Mann   

       popularity  duration_ms  explicit  \
86542          48       273937         0   
86436          45       260520         0   
88091          64       157507         1   
89074          52       209259         1   
86177          37       260349         0   

                                                artists  \
86542                                  [Calema, Batuta]   
86436                        [Brandt, Robin Stjernberg]   
88091                                       [A$AP Ferg]   
89074  [Don Joe, Ketama126, Franco126, Franco Califano] 

Let's check out our data!

In [16]:
print(tracks_not100_sub.info())
tracks_not100_sub.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                1100 non-null   object        
 1   name              1100 non-null   object        
 2   popularity        1100 non-null   object        
 3   duration_ms       1100 non-null   object        
 4   explicit          1100 non-null   object        
 5   artists           1100 non-null   object        
 6   id_artists        1100 non-null   object        
 7   release_date      1100 non-null   datetime64[ns]
 8   danceability      1100 non-null   float64       
 9   energy            1100 non-null   float64       
 10  key               1100 non-null   object        
 11  loudness          1100 non-null   float64       
 12  mode              1100 non-null   object        
 13  speechiness       1100 non-null   float64       
 14  acousticness      1100 n

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,main_artist,duration_min,hot100
0,6GQLX6Z28fYwDNCrhaKzYF,Undercover Martyn,68,166573,0,[Two Door Cinema Club],['536BYVgOnRky0xjsPT96zl'],2010-03-01,0.446,0.871,...,0.1160,0.000252,0.000008,0.0996,0.612,159.863,4,Two Door Cinema Club,2.776217,False
1,6O4QbnxKqUCJMIGJe6rTaN,"Pirouette, cacahouète",44,171147,0,[Chansons Et Comptines],['4rsK66w6MjGtwI5QnhFt0P'],2010-03-21,0.783,0.619,...,0.0286,0.492000,0.000000,0.0613,0.769,140.130,4,Chansons Et Comptines,2.852450,False
2,04J24gXLrUEYyNmmoJMLNa,Make A Circuit Me,0,176427,0,[The Polecats],['0ySLj2XiEOFtixInK1GAD4'],2010-12-21,0.353,0.953,...,0.2070,0.000397,0.076300,0.1810,0.545,188.355,4,The Polecats,2.940450,False
3,1PdnoVxzvexOoLkOaTq7KX,Poupata,29,214427,0,"[Michal David, Kroky Frantiska Janecka]","['6rA1HqwDyDDRcq6WHTwr2G', '2wMHvKKOWjwp77Qghe...",2010-06-30,0.580,0.694,...,0.0397,0.080400,0.000000,0.0903,0.887,139.607,4,Michal David,3.573783,False
4,3DOOQAScRTa3oTszGN3XZg,Todo vuelve a su lugar,48,299973,0,"[Fidel Nadal, I Nesta]","['7l2wptrNdI4wEHxTlWa6SS', '2LraTaRLVho6BWao5J...",2010-11-08,0.624,0.892,...,0.0622,0.112000,0.000000,0.0816,0.777,77.974,4,Fidel Nadal,4.999550,False


Great! Let's save both the datasets!

In [17]:
tracks_not100.to_csv('tracks_not100.csv')
tracks_not100_sub.to_csv('tracks_not100_sub.csv')

And that's that! Next notebook we will start to play around with these datasets!

Written by Mir Adnan Mahmood for the Erdos Institute DataScience Bootcamp Project.